This notebook does basically the same thing as [Raster Analysis](https://github.com/willgeary/anomaly_detection_raster_images/blob/master/Raster%20Analysis.ipynb) except now we are using the [rasterio](https://mapbox.github.io/rasterio/) library to read and write georeferenced tiff files.

# Import libraries

In [1]:
import rasterio
import numpy as np
import pandas as pd
import cv2

# 1) Process Gridded Population of the World (GPW) raster

### Read in GPW raster

In [2]:
GPW_PATH = '../raster/Resampled_BlackMarble_GPW/gpw_v4_population_count_20151.tif'

In [3]:
with rasterio.open(GPW_PATH) as src:
    
    # The profile contains the metadata
    GPW_profile = src.profile
    
    # Read the first and only band into an array
    GPW = src.read(1)

View raster metadata in GPW:

In [4]:
GPW_profile

{'affine': Affine(0.00833333333333339, 0.0, -180.0,
       0.0, -0.00833333333333339, 85.0000000000092),
 'blockxsize': 128,
 'blockysize': 128,
 'count': 1,
 'crs': CRS({'init': u'epsg:4326'}),
 'driver': u'GTiff',
 'dtype': 'float32',
 'height': 17400,
 'interleave': 'band',
 'nodata': -3.402823e+38,
 'tiled': True,
 'transform': (-180.0,
  0.00833333333333339,
  0.0,
  85.0000000000092,
  0.0,
  -0.00833333333333339),
 'width': 43200}

Note the `nodata` field :

Pixels marked as having "no data" (such as oceans or lakes) are replaced with a huge negative number `-3.402823e+38`.

We want to keep all of our pixels, so we will deal with this later by setting the value of these `nodata` pixels to zero.

Also note `dtype: float32`. We want to work with 64 bit floats, so the first thing we need to do is change these to 64 bit floats, which is numpy's default

###  Convert GPW to 64 bit floats

In [5]:
GPW = GPW.astype(float) # default float is 64-bit in numpy 
GPW_profile.update(dtype='float64') # need to update profile anytime metadata changes

### Replace huge negative nodata values in GPW with zeros

In [6]:
GPW[ GPW < -10000000000 ] = 0

### Remove the concept of `nodata` altogether, so that all pixels are kept intact on output

In [7]:
del GPW_profile['nodata']

### Scale GPW from 0 to 1

In [8]:
def min_max_scaler(array):
    array_scaled = (array - array.min()) / (array.max() - array.min())
    return array_scaled

In [9]:
GPW_scaled = min_max_scaler(GPW)

### Smooth GPW with Gaussian Blur

In [10]:
kernel_size = (15,15)

In [11]:
GPW_gaussian = cv2.GaussianBlur(GPW_scaled, kernel_size,0)

### Enhance contrast in GPW

http://scikit-image.org/docs/dev/auto_examples/color_exposure/plot_equalize.html

In [12]:
from skimage import exposure

In [13]:
# Cumulative count cut
p_min, p_max = 0.75, 99.25

# Rescale intensity to enhance contrast
pmin, pmax = np.percentile(GPW_gaussian, (p_min, p_max))
GPW_gaussian_enhanced = exposure.rescale_intensity(GPW_gaussian, in_range=(pmin, pmax))

### Scale GPW back to 0 to 255

In [14]:
GPW_final = GPW_gaussian_enhanced * 255

### Convert GPW to integers

In [15]:
GPW_final = np.uint8(GPW_final)
GPW_final_profile = GPW_profile
GPW_final_profile.update(dtype=rasterio.uint8) # need to update profile anytime metadata changes

### Write Processed GPW to GeoTIFF

In [16]:
with rasterio.open('GPW_smoothed_enhanced.tif', 'w', **GPW_final_profile) as dst:
    dst.write(GPW_final, 1)

/anaconda/lib/python2.7/site-packages/rasterio/__init__.py:160: FutureWarning: GDAL-style transforms are deprecated and will not be supported in Rasterio 1.0.
  transform = guard_transform(transform)


### Delete the intermediate arrays to save memory

In [17]:
del GPW_gaussian
del GPW_scaled

# 2) Process Black Marble (BM) raster

### Read in Black Marble raster

In [18]:
BM_PATH = "../raster/Resampled_BlackMarble_GPW/BlackMarble_mosaic_resample_32BIT_normalize.tif"

In [19]:
with rasterio.open(BM_PATH) as src:
    
    # The profile contains the metadata
    BM_profile = src.profile
    
    # Read the first and only band into an array
    BM = src.read(1)

View the raster metadata:

In [20]:
BM_profile

{'affine': Affine(0.00833333333333339, 0.0, -180.0,
       0.0, -0.00833333333333339, 85.00000000000001),
 'blockxsize': 128,
 'blockysize': 128,
 'count': 1,
 'crs': CRS({}),
 'driver': u'GTiff',
 'dtype': 'float32',
 'height': 17400,
 'interleave': 'band',
 'nodata': -3.402823e+38,
 'tiled': True,
 'transform': (-180.0,
  0.00833333333333339,
  0.0,
  85.00000000000001,
  0.0,
  -0.00833333333333339),
 'width': 43200}

### Convert Black Marble to 64 bit floats

In [21]:
BM = BM.astype(float)
BM_profile.update(dtype='float64') # need to update profile anytime metadata changes

### Replace negative nodata values in Black Marble with zeros

In [22]:
BM[ BM < -10000000000 ] = 0

### Remove the concept of nodata so all pixels are kept on output

In [23]:
del BM_profile['nodata']

### Scale Black Marble from 0 to 1

In [24]:
BM_scaled = min_max_scaler(BM)

### Smooth Black Marble with Gaussian Blur

In [25]:
kernel_size = (11,11)

In [26]:
BM_gaussian = cv2.GaussianBlur(BM_scaled, kernel_size,0)

### Enhance contrast in Black Marble

http://scikit-image.org/docs/dev/auto_examples/color_exposure/plot_equalize.html

In [27]:
# Cumulative count cut
p_min, p_max = 0.075, 99.925

# Rescale intensity to enhance contrast
pmin, pmax = np.percentile(BM_gaussian, (p_min, p_max))
BM_gaussian_enhanced = exposure.rescale_intensity(BM_gaussian, in_range=(pmin, pmax))

### Scale Black Marble back to 0 to 255

In [28]:
BM_final = BM_gaussian_enhanced * 255

### Convert Black Marble to integers

In [29]:
BM_final = np.uint8(BM_final)

### Write Processed Black Marble to GeoTIFF

In [30]:
# need to update profile anytime metadata changes
BM_final_profile = BM_profile
BM_final_profile.update(dtype=rasterio.uint8)

with rasterio.open('BM_smoothed_enhanced.tif', 'w', **BM_final_profile) as dst:
    dst.write(BM_final, 1)

### Delete the intermediate arrays to save memory

In [31]:
del BM_gaussian
del BM_scaled

# 3) Calculate difference

Gridded Population Minus Black Marble

In [32]:
diff = GPW_gaussian_enhanced - BM_gaussian_enhanced

Save the difference array to a geotiff.

In [34]:
# need to update profile anytime metadata changes
diff_profile = BM_profile
diff_profile.update(dtype='float64')

with rasterio.open('Difference_smoothed_enhanced.tif', 'w', **diff_profile) as dst:
    dst.write(diff, 1)